# グループチャットオーケストレーションシステム

## 概要

このノートブックでは、Semantic Kernel のグループチャット機能を活用して複数エージェントの協調作業システムを構築する方法を学習します。

![alt text](./../../docs/img/image-01-03.png)

### 主な学習内容
- Group Chat マネージャの処理フロー
- RoundRobinGroupChatManager による順次実行制御
- カスタム Group CHat マネージャによる柔軟な制御フロー（次章 05 で紹介）

### 本章で作る構成
- リフレクションによるレポート作成
    - Witer：Microsoft Docs MCP プラグインを持つ執筆担当 
    - Reviewer：レポートを包括的にレビューする担当

### 実用的な活用場面

- 技術ドキュメントの調査・要約
- 情報の正確性チェックと品質管理
- 複数の観点からの検証プロセス

# ライブラリのインポート

グループチャットオーケストレーションシステムの構築に必要なライブラリをインポートします。

## 主要コンポーネント

- **GroupChatOrchestration**: 複数エージェントの協調作業を管理
- **RoundRobinGroupChatManager**: エージェント間の順次実行を制御
- **MCPStreamableHttpPlugin**: Microsoft Learn ドキュメントへのアクセス

In [10]:
import asyncio
import os
from dotenv import load_dotenv

from semantic_kernel.agents import Agent, ChatCompletionAgent, GroupChatOrchestration, RoundRobinGroupChatManager
from semantic_kernel.agents.runtime import InProcessRuntime
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents import ChatMessageContent

from semantic_kernel.contents import (
    FunctionCallContent, FunctionResultContent
)
from semantic_kernel.connectors.mcp import MCPStreamableHttpPlugin

# 環境変数の取得

Azure OpenAI Service への接続に必要な認証情報を環境変数から取得します。

In [11]:
load_dotenv(override=True)

AZURE_OPENAI_ENDPOINT=os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY=os.getenv("AZURE_OPENAI_API_KEY")
AZURE_DEPLOYMENT_NAME=os.getenv("AZURE_DEPLOYMENT_NAME")

# プラグイン

Microsoft Learn の公式ドキュメントにアクセスするための MCP プラグインを作成します。

## Microsoft Learn MCP プラグイン

信頼性の高い技術情報を検索・取得するためのプラグインです。

In [ ]:
# MCPのエンドポイントURL
MCP_URL = "https://learn.microsoft.com/api/mcp/"


microsoft_docs_plugin = MCPStreamableHttpPlugin(
    name="microsoft_docs",
    url=MCP_URL
)

await microsoft_docs_plugin.connect()

# クライアントの初期化

Azure OpenAI Service への接続クライアントを初期化します。

In [13]:
# Chat Completion API クライアントの初期化
azure_completion_service  = AzureChatCompletion(
    service_id="azure_completion_agent",
    deployment_name=AZURE_DEPLOYMENT_NAME,
    endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY
)

# エージェントの作成

協調作業を行う専門エージェントを作成します。

## 役割分担

- **MSLearnReporter**: MS Learn ドキュメントの検索・要約・レポート作成
- **MSLearnReviewer**: レポート内容の品質チェック・フィードバック提供

In [ ]:
# ---Writer エージェント---
writer = ChatCompletionAgent(
    name="MSLearnReporter",
    description="MS Learnドキュメントの要約・レポート専門エージェント",
    instructions=(
        "あなたはMicrosoft Learnの公式ドキュメントを検索・要約し、"
        "ユーザーの質問やテーマに合わせて正確かつ簡潔なレポートを作成してください。"
        "レビュワーからフィードバックや修正指示があった場合は、その内容を真摯に受け止め、"
        "指摘された点を修正したうえで再度レポートを提出してください。"
        "常に最新かつ信頼性の高い情報を参照し、文献やURLがある場合は必ず明記してください。"
        "回答は日本語で、箇条書き・要点整理を意識し、マークダウンで出力ください。"
    ),
    service=azure_completion_service,
    plugins=[microsoft_docs_plugin],
)

# --- Reviewer エージェント ---
reviewer = ChatCompletionAgent(
    name="MSLearnReviewer",
    description="MS Learnレポート内容のレビュー・フィードバック専門エージェント",
    instructions=(
        "あなたはMS Learnレポートの品質管理担当です。"
        "MSLearnReporterが作成したレポートをチェックし、"
        "・内容の正確性や網羅性"
        "・説明の分かりやすさ"
        "・論点の抜け漏れや参考リンクの妥当性"
        "などを厳しくレビューしてください。"
        "改善点や追加してほしいポイントがあれば、具体的かつ丁寧に指摘し、"
        "必ず日本語でコメントとして返してください。"
    ),
    service=azure_completion_service,
)


# オーケストレーションの作成

複数エージェントの協調作業を管理するグループチャットオーケストレーションを設定します。

## 実行フロー

1. Writer エージェントがトピックを調査・レポート作成
2. Reviewer エージェントが内容をチェック・フィードバック
3. 最大3ラウンドまで改善を繰り返し
4. 最終レポートを Markdown ファイルとして出力

In [15]:
def agent_response_callback(message: ChatMessageContent) -> None:
    """エージェントが応答を生成したときに呼び出されるコールバック関数"""
    print(f"{message.name}: {message.content}")
    for item in message.items:
        if isinstance(item, FunctionCallContent):
            print(f"Calling '{item.name}' with arguments '{item.arguments}'")
        if isinstance(item, FunctionResultContent):
            print(f"Result from '{item.name}' is '{item.result}'")

In [17]:
# async def run_group_chat():
# オーケストレーションを構成
group_chat_orchestration = GroupChatOrchestration(
    members=[writer, reviewer],
    manager=RoundRobinGroupChatManager(max_rounds=3),
    agent_response_callback=agent_response_callback,
)

# ランタイムを初期化＆開始
runtime = InProcessRuntime()
runtime.start()

# タスクを実行
orchestration_result = await group_chat_orchestration.invoke(
    task="Micorosoft の AI エージェント開発手法",
    runtime=runtime,
)

# 結果を取得
value = await orchestration_result.get()
# print(f"# Result\n{value}")

# 出力をMarkdown形式で保存
output_dir = "./output_report"
os.makedirs(output_dir, exist_ok=True)

output_path = os.path.join(output_dir, "report.md")
markdown_content = value if isinstance(value, str) else str(value)
with open(output_path, "w", encoding="utf-8") as f:
    f.write(markdown_content)

print(f"Markdownファイルとして保存しました: {output_path}")

# ランタイムを停止
await runtime.stop_when_idle()

MSLearnReporter: 
Calling 'microsoft_docs-microsoft_docs_search' with arguments '{"question":"Microsoft AI エージェント 開発手法"}'
MSLearnReporter: 
Result from 'microsoft_docs-microsoft_docs_search' is '[TextContent(inner_content=TextContent(type='text', text='[{"title":"Custom engine agents for Microsoft 365 overview","content":"# Custom engine agents for Microsoft 365 overview\\n## Development approaches for custom engine agents\\n| Scenario | Description | Recommended approach| \\n|  --- | --- | ---  |\\n| **Legal case analysis** | A law firm creates a standalone AI agent using Azure AI Foundry. The agent uses a custom-trained LLM for case law analysis and integrates with external legal databases. The agent is used in the firm\\u0027s case management system but should also be accessible within Microsoft 365 Copilot and have access to documents in SharePoint. | Because the agent runs in both the law firm\\u0027s internal case management system (a custom-built app) and Microsoft 365, use the 